# 实验参数设置

In [1]:
from FederatedLearning.learner import CNNModel

clients_num = 10 # 客户端总数
rotation_cycle = 5 # 角色轮换间隔轮数
max_round = 20 # 联邦学习轮数
learning_rate = 0.01
model_class = CNNModel # 选用CNN模型

## 数据切分

In [2]:
import data_split
# 使用data_split模块中的函数获取PyTorch数据集
train_dataset = data_split.get_mnist_pytorch_dataset(train=True)
test_dataset = data_split.get_mnist_pytorch_dataset(train=False)
# test
# 使用正确的数据集对象创建数据加载器
client_dataloaders = data_split.create_client_dataloaders(train_dataset, clients_num // 2, 64, num_workers=clients_num) * 2
client_test_loader = data_split.create_client_dataloaders(test_dataset, clients_num // 2, 64) * 2

2025-10-23 16:29:09,013 - DGS_BCFL - INFO - 
切分5份数据集...
2025-10-23 16:29:09,015 - DGS_BCFL - INFO -  1 数据准备完成，样本数: 12000
2025-10-23 16:29:09,017 - DGS_BCFL - INFO -  2 数据准备完成，样本数: 12000
2025-10-23 16:29:09,018 - DGS_BCFL - INFO -  3 数据准备完成，样本数: 12000
2025-10-23 16:29:09,019 - DGS_BCFL - INFO -  4 数据准备完成，样本数: 12000
2025-10-23 16:29:09,020 - DGS_BCFL - INFO -  5 数据准备完成，样本数: 12000
2025-10-23 16:29:09,021 - DGS_BCFL - INFO - 
切分5份数据集...
2025-10-23 16:29:09,022 - DGS_BCFL - INFO -  1 数据准备完成，样本数: 2000
2025-10-23 16:29:09,023 - DGS_BCFL - INFO -  2 数据准备完成，样本数: 2000
2025-10-23 16:29:09,023 - DGS_BCFL - INFO -  3 数据准备完成，样本数: 2000
2025-10-23 16:29:09,024 - DGS_BCFL - INFO -  4 数据准备完成，样本数: 2000
2025-10-23 16:29:09,024 - DGS_BCFL - INFO -  5 数据准备完成，样本数: 2000


## 开始训练
### 无恶意客户端训练

In [ ]:
from owner import Owner
from client import Client, BadClient
    
    
# 初始化管理者，并获得初始化字典
owner = Owner(rotation_cycle=1, model_class=model_class)
main_dict = owner.get_main_dict()

# 初始化客户端以及身份
clients = [owner]
for i in range(clients_num):
    client_name = f"client_{i + 1}"
    client = Client(epochs=2, client_name=client_name, data_loader=client_dataloaders[i],
                    test_loader=client_test_loader[i], ModelClass=model_class, main_dict=main_dict, learning_rate=learning_rate)
    owner.join(client_name)
    clients.append(client)

import threading

for _ in range(max_round):
    t = [threading.Thread(target=client.run) for client in clients]
    _ = [i.start() for i in t]
    _ = [i.join() for i in t]

    


# 保存实验过程和结果
import pickle
with open(f"main_dict_{clients_num}_{0}.pkl", "wb") as f:
    pickle.dump(main_dict, f)

### 有恶意客户端训练
#### 恶意客户端 占比 10%

In [ ]:
from owner import Owner
from client import Client, BadClient

bad_percent = 0.1
bad_num = int(clients_num * bad_percent)
    
# 初始化管理者，并获得初始化字典
owner = Owner(rotation_cycle=1, model_class=model_class)
main_dict = owner.get_main_dict()

# 初始化客户端以及身份
clients = [owner]
for i in range(clients_num-bad_num):
    client_name = f"client_{i + 1}"
    client = Client(epochs=2, client_name=client_name, data_loader=client_dataloaders[i],
                    test_loader=client_test_loader[i], ModelClass=model_class, main_dict=main_dict, learning_rate=learning_rate)
    owner.join(client_name)
    clients.append(client)
for i in range(bad_num):
    client_name = f"bad_client_{i + 1}"
    client = BadClient(epochs=2, client_name=client_name, data_loader=client_dataloaders[i],
                    test_loader=client_test_loader[i], ModelClass=model_class, main_dict=main_dict, learning_rate=learning_rate)
    owner.join(client_name)
    clients.append(client)
    
import threading

for _ in range(max_round):
    t = [threading.Thread(target=client.run) for client in clients]
    _ = [i.start() for i in t]
    _ = [i.join() for i in t]

    


# 保存实验过程和结果
import pickle
with open(f"main_dict_{clients_num}_{bad_percent}.pkl", "wb") as f:
    pickle.dump(main_dict, f)

2025-10-23 16:29:13,838 - DGS_BCFL - INFO - Owner开始初始化 角色
2025-10-23 16:29:13,839 - DGS_BCFL - INFO - [client_1]  round 1 等待角色分配...
2025-10-23 16:29:13,840 - DGS_BCFL - INFO - [client_2] 当前轮次角色: validator
2025-10-23 16:29:13,841 - DGS_BCFL - INFO - [client_3] 当前轮次角色: validator
2025-10-23 16:29:13,842 - DGS_BCFL - INFO - [client_4] 当前轮次角色: validator
2025-10-23 16:29:13,844 - DGS_BCFL - INFO - [client_5] 当前轮次角色: learner
2025-10-23 16:29:13,846 - DGS_BCFL - INFO - [client_6] 当前轮次角色: learner
2025-10-23 16:29:13,846 - DGS_BCFL - INFO - [client_7] 当前轮次角色: learner
2025-10-23 16:29:13,848 - DGS_BCFL - INFO - [client_8] 当前轮次角色: learner
2025-10-23 16:29:13,850 - DGS_BCFL - INFO - [client_9] 当前轮次角色: learner
2025-10-23 16:29:13,852 - DGS_BCFL - INFO - [bad_client_1] 当前轮次角色: learner
2025-10-23 16:29:13,873 - DGS_BCFL - INFO - [client_2] 验证者初始化...
2025-10-23 16:29:13,874 - DGS_BCFL - INFO - [client_4] 验证者初始化...
2025-10-23 16:29:13,876 - DGS_BCFL - INFO - [client_3] 验证者初始化...
2025-10-23 16:29:14,071 

## 计算结果可视化

In [ ]:
import pickle
from matplotlib import pyplot as plt
import numpy as np
# 读取实验结果
main_dict = pickle.load(open("main_dict.pkl", "rb"))

# 精确度变化可视化
global_accuracy_history = main_dict["global_accuracy_history"]
# 创建图形
plt.figure(figsize=(12, 7))
plt.ylim(0, 100)
# 绘制主要曲线
epochs = list(range(1, len(global_accuracy_history) + 1))
plt.plot(epochs, global_accuracy_history, 'b-', linewidth=1, marker='o', markersize=6, 
         markerfacecolor='red', markeredgecolor='darkred', markeredgewidth=1, 
         label='模型准确度')
# 显示图表
plt.show()
    


